In [111]:
import sys
sys.path.append("../..")

import os
import pickle
import random

from copy import deepcopy

import pandas as pd
from pandas import DataFrame

from app.components.extract_data.extract_data import getDfPerson, getDfConsumo, getDictV9001toGroupPt, getDictV9001toGroupEn
from app.components.simple_types import Nutrition, State
from app.components.search.papa_search import SearchResult, papaSingleSeach, fitness, SearchResult
from typing import Callable

from app.components.extract_data.extract_data import (
    getDictPersonIdStrata,
    getDictStrataMeals,
)

In [112]:
datasetPath = os.getcwd() + "/../../datasets/"
datasetPicklePath = os.getcwd() + "/../../datasets/pickle/"

In [113]:
dfConsumo = getDfConsumo()
dfPerson = getDfPerson()

In [114]:
len(dfPerson["ESTRATO_POF"].unique())

575

In [115]:
class PersonInformation:
    def __init__(self, person:str):
        # Id da pessoa no POF
        self.id:str = person
        # Dieta inicial da pessoa
        self.initialDiet:State = State.getStateByPersonId(person)        
        # Dieta inicial do passo atual
        self.actualDiet:State = State.getStateByPersonId(person) 
        # Nutriçao recomendada para a pessoa
        self.targetNutrition= Nutrition.idealNutritionByPersonId(person) 

In [116]:
class DietAgent:
    def __init__(self, person:str):
        self.personInformation:PersonInformation = PersonInformation(person)
        self.optimizedDiets:list[State] = [State.getStateByPersonId(person)]

In [117]:
class DietAllyEnemy:
    def __init__(self, person:str):
        self.enemy:int = -1
        self.ally:int = -1
        self.bestDiet:State = State.getStateByPersonId(person) 
        self.bestDietFitness:float = None
    

In [118]:
class HealthAgent:
    def __init__(self):
        self.personDietAllyEnemmy:dict[str, DietAllyEnemy] = {}
        self.dietAgents:dict[str, DietAgent] = {}
        
    def add_person(self, person:str): 
        self.personDietAllyEnemmy[person] = DietAllyEnemy(person)
        self.dietAgents[person] = DietAgent(person)
        
    def assign_social(self):
        allies = list(self.dietAgents.keys())
        enemies = list(self.dietAgents.keys())
        random.shuffle( allies )
        random.shuffle( enemies )
        
        people = list(self.dietAgents.keys())
        for id, person in enumerate(people):
            self.personDietAllyEnemmy[person].ally = allies[id]
            self.personDietAllyEnemmy[person].enemy = enemies[id]
            
    def updateBest(self, person:str, candidateBest:State, nutritionFactor:float=1.2, distanceFactor:float=0.1):
        self.personDietAllyEnemmy[person]
        
        if self.personDietAllyEnemmy[person].bestDietFitness == None:
            self.personDietAllyEnemmy[person].bestDietFitness = fitness(
                state=self.personDietAllyEnemmy[person].bestDiet,
                initialState=self.dietAgents[person].personInformation.actualDiet,
                targetNutrition=self.dietAgents[person].personInformation.targetNutrition,
                nutritionFactor=nutritionFactor,
                distanceFactor=distanceFactor, 
            )
        
        candidateFitness = fitness(
            state=candidateBest,
            initialState=self.dietAgents[person].personInformation.actualDiet,
            targetNutrition=self.dietAgents[person].personInformation.targetNutrition,
            nutritionFactor=nutritionFactor,
            distanceFactor=distanceFactor, 
        )
        
        if( self.personDietAllyEnemmy[person].bestDietFitness > candidateFitness ):
            self.personDietAllyEnemmy[person].bestDietFitness = candidateFitness
            self.personDietAllyEnemmy[person].bestDiet = candidateBest
    
    def get_diet_allyDiet_enemyDiet(self, person:str):
        ally = self.personDietAllyEnemmy[person].ally
        enemmy = self.personDietAllyEnemmy[person].ally
        
        diets = self.dietAgents[person].optimizedDiets
        dietsAlly = self.dietAgents[ally].optimizedDiets  
        dietsEnemy = self.dietAgents[enemmy].optimizedDiets
        
        return diets, dietsAlly, dietsEnemy
    
    def hero(self, person:str):
        diets, dietsAlly, dietsEnemy = self.get_diet_allyDiet_enemyDiet(person)
        strata = getDictPersonIdStrata().get(person, None)
        mealList = getDictStrataMeals()[strata]
        
        newDiets = []
        for diet in diets:
            dietAlly = random.choice(dietsAlly)
            dietEnemy = random.choice(dietsEnemy)
            
            newDiet = State(diet.data)
            for meal in mealList:
                if( dietAlly[meal]>0.0 ):
                    newDiet[meal] = (newDiet[meal]+dietAlly[meal]+dietEnemy[meal])/3.0
            
            newDiets.append(newDiet)
            
        self.dietAgents[person].optimizedDiets.extend(newDiets)
        return newDiets
    
    def coward(self, person:str): 
        diets, dietsAlly, dietsEnemy = self.get_diet_allyDiet_enemyDiet(person)
        strata = getDictPersonIdStrata().get(person, None)
        mealList = getDictStrataMeals()[strata]
        
        newDiets = []
        for diet in diets:
            dietAlly = random.choice(dietsAlly)
            dietEnemy = random.choice(dietsEnemy)
            
            newDiet = State(diet.data)
            for meal in mealList:
                if( abs(diet[meal]-dietAlly[meal]) < abs(diet[meal]-dietEnemy[meal])):
                    newDiet[meal] = (newDiet[meal]+dietAlly[meal])/2.0
                else:
                    newDiet[meal] = dietAlly[meal]
            newDiets.append(newDiet)
            
        self.dietAgents[person].optimizedDiets.extend(newDiets)
        return newDiets

In [119]:
personID = "110000016#7#1#5"

In [120]:
healthAgent = HealthAgent()
dietAgent = DietAgent(personID)

In [121]:
healthAgent.add_person(personID)

In [ ]:
#Find people 
dfPerson = getDfPerson()
stratos = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="male") ]["ESTRATO_POF"].to_list()
quantity = 100
male25 = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="male") ]["PESSOA"].to_list()[:quantity]
male60 = dfPerson[ (dfPerson["AGE"]==60) & (dfPerson["GENDER"]=="male") ]["PESSOA"].to_list()[:quantity]
female25 = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="female") ]["PESSOA"].to_list()[:quantity]
female60 = dfPerson[ (dfPerson["AGE"]==60) & (dfPerson["GENDER"]=="female") ]["PESSOA"].to_list()[:quantity]

In [ ]:
def multiAgentSearch(
        people:list[str], 
        resultFileName:str, 
        maxInterations:int = 3,
        maxSteps:int = 10,
        maxSearchSteps:int = 10,
        nutritionFactor:float = 1.2,
        distanceFactor:float = 0.1, 
        verbose:bool=False) -> dict[str, list[SearchResult]]:
    try:
        with open("./output/" + resultFileName, "rb") as file:
            return pickle.load(file)
    except:
        
        healthAgent = HealthAgent()
        for person in people:
            healthAgent.add_person(person)
        healthAgent.assign_social()
        
        results:dict[str, list[SearchResult]] = {}
        
        phases = ["corward", "hero"]
        
        for interation in range(maxInterations):
            print("interation:", interation)
            
            # RUN multiAgentSearch
            for step in range(maxSteps):
                                
                for agent in healthAgent.dietAgents.values():
                    person = agent.personInformation.id
                    
                    # Interação Social
                    if phases[step%2] == "corward":
                        healthAgent.coward(person)
                    elif phases[step%2] == "hero":
                        healthAgent.hero(person)
          
                    result = papaSingleSeach(
                                personID=person, 
                                verbose=False, 
                                unit=50, 
                                max_unit=3, 
                                max_population_set=20, 
                                max_population_selected=10, 
                                expansion_set=20, 
                                expansion_select=5, 
                                max_steps=maxSearchSteps, 
                                nutritionFitness=Nutrition.absDifferenceNegativePenalty, 
                                nutritionFactor=nutritionFactor,
                                distanceFitness=State.squareDifference,
                                distanceFactor = distanceFactor,
                                preserve_best=True,
                                crossover=0.1, 
                                initialState=agent.personInformation.actualDiet,
                                initialPopulation=agent.optimizedDiets,
                            )

                    healthAgent.updateBest(
                            person=person,
                            candidateBest=result.finalMeal,
                        )
                        
                       
            if verbose:
                print("### Interation", interation, "####\n")
                
            for agent in healthAgent.dietAgents.values():
                person = agent.personInformation.id 
                
                if verbose:
                    print("Agent:", person)
                    print("Initial diet:", agent.personInformation.actualDiet)
                    print("Final diet:", healthAgent.personDietAllyEnemmy[person].bestDiet)
                
                initialFitness = fitness(
                    agent.personInformation.actualDiet,
                    agent.personInformation.actualDiet,
                    agent.personInformation.targetNutrition,
                    nutritionFactor=nutritionFactor,
                    distanceFactor=distanceFactor,
                )
                
                if verbose:
                    print("Initial fitness:", initialFitness)
                    print("Final fitness:", healthAgent.personDietAllyEnemmy[person].bestDietFitness)
                    print("#############")
                
                if results.get(person, None) == None:
                    results[person] = list()
                
                results[person].append(
                            SearchResult(
                                [person],
                                agent.personInformation.actualDiet,
                                healthAgent.personDietAllyEnemmy[person].bestDiet,
                            )
                        )
                
                agent.personInformation.actualDiet = deepcopy(healthAgent.personDietAllyEnemmy[person].bestDiet)
                
            
        with open("./output/" + resultFileName, "wb") as file:
            pickle.dump(results, file) 
        
        return results
            

In [ ]:

groups = [male25, female25, male60, female60] 
fileNames = [
    "male25_100_agent.pickle",
    "female25_100_agent.pickle",
    "male60_100_agent.pickle",
    "female60_100_agent.pickle"
]
resultsArray:list[dict[str, list[SearchResult]]]= [
    {},{},{},{}
]

for i, group in enumerate(groups):
    fileName = fileNames[i]
    print("Starting:", fileName)

    resultsArray[i] = multiAgentSearch(
                    people=group, 
                    resultFileName=fileName, 
                    maxInterations=4, 
                    maxSteps=10, 
                    maxSearchSteps=5,
                    verbose=False
                )

Starting: male25_100_agent.pickle
Starting: female25_100_agent.pickle
Starting: male60_100_agent.pickle
Starting: female60_100_agent.pickle


In [140]:
for personResult in resultsArray:
    for person, results in personResult.items():
        print("Person:", person)
        targetNutrition = Nutrition.idealNutritionByPersonId(person) 
        fitnessValues = []
        fitnessValues.append(fitness(
                        results[0].initialMeal,
                        results[0].initialMeal,
                        targetNutrition,
                        nutritionFactor=1.2,
                        distanceFactor=0.1,
                    ))
        
        for result in results:
            fitnessValues.append(fitness(
                        result.initialMeal,
                        result.finalMeal,
                        targetNutrition,
                        nutritionFactor=1.2,
                        distanceFactor=0.1,
                    ))
            
        print(fitnessValues)

Person: 110004584#15#1#1
[4668.263681910302, 6060.571374217994, 2132.4813055979303, 736.1321321165658, 661.2439021343677]
Person: 110010188#3#1#4
[7501.0536221218745, 9808.745929814182, 3206.2208978690473, 1511.5256987063324, 1124.9341313463542]
Person: 110012387#6#1#3
[4612.694675254262, 5268.386982946569, 2410.312806304838, 1662.512000326781, 1247.782379067193]
Person: 110004584#15#1#1
[4668.263681910302, 5868.263681910302, 2080.808894247472, 1050.4786025719998, 848.9540995740292]
Person: 110010188#3#1#4
[7501.0536221218745, 10001.053622121874, 3222.6206746904686, 975.6134922790834, 975.6134922790834]
Person: 110012387#6#1#3
[4612.694675254262, 5189.617752177339, 2285.0453364883747, 1724.7442566753268, 1724.7442566753268]
Person: 110004584#15#1#1
[4668.263681910302, 6060.571374217994, 2123.735122538401, 736.1321321165658, 661.2439021343677]
Person: 110010188#3#1#4
[7501.0536221218745, 9424.130545198797, 3684.40563543615, 1329.9138638534012, 1329.9138638534012]
Person: 110012387#6#1#3

In [143]:
print(len(resultsArray[1]))

3
